In [1]:
from tensorflow.keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras import Sequential
from data import Data

In [2]:
# data

data = Data("data")
data.config_iter(64,4)

In [3]:
%%time
res = 0
for x,x2,y in data:
    print(x.shape)
    res += x.shape[0]
res

(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 256)
(64, 256, 

39778